<a href="https://colab.research.google.com/github/minoxd/fiber_optic_box_inspection/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environmental Setup

## mount gdrive, import libraries, declare constants, specify paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from pathlib import Path
import shutil
import tarfile
import torch

if Path('/content/sample_data').exists():
    shutil.rmtree('/content/sample_data')

In [ ]:
IMGSZ = 960

These files are required in the `Data_INF` folder:
- data_label_All.tar.gz
- inf_raw_classes_v1.0.json
- matching_name_id_v1.0.xlsx

In [ ]:
BASE_DIR = '/content/drive/MyDrive/fiber_optic_box_inspection'
if not Path(BASE_DIR).exists():
    raise FileNotFoundError(f'Base directory {BASE_DIR} not found. Check Google Drive mount.')

LOCAL_BASE_DIR = '/content'

paths = {
    'base_dir':                                   Path(BASE_DIR),
#     'Data_INF':                                   Path(BASE_DIR, 'Data_INF'),
    'dataset_dir':                                Path(BASE_DIR, 'dataset_v2_ALL'),
    'yolo_output_dir':                            Path(BASE_DIR, 'runs_yolo'),
}

# if not paths['dataset_dir'].exists():
#     Path.mkdir(paths['dataset_dir'])

# paths['raw_classes'] =                            Path(paths['Data_INF'], 'inf_raw_classes_v1.0.json')
# paths['new_raw_classes'] =                        Path(paths['Data_INF'], 'inf_raw_classes_v1.0_new.json')
# paths['matching_name_id'] =                       Path(paths['Data_INF'], 'matching_name_id_v1.0.xlsx')
# paths['dataset_compressed'] =                     Path(paths['Data_INF'], 'data_label_ALL.tar.gz')

paths['processed_dataset_compressed'] =           Path(paths['dataset_dir'], 'processed_dataset.tar.gz')

local_paths = {
    'base_dir':                                   Path(LOCAL_BASE_DIR),
    'dataset_dir':                                Path(LOCAL_BASE_DIR, 'dataset_v2_ALL'),
}

if not local_paths['dataset_dir'].exists():
    Path.mkdir(local_paths['dataset_dir'])

# local_paths['image_label_dir'] =                  Path(local_paths['dataset_dir'], 'image_label')
# local_paths['label_bbox_backup_dir'] =            Path(local_paths['dataset_dir'], 'label_bbox_backup')
# local_paths['label_id_backup_dir'] =              Path(local_paths['dataset_dir'], 'label_id_backup')
# local_paths['processed_dataset_compressed'] =     Path(local_paths['dataset_dir'], 'processed_dataset.tar.gz')
# local_paths['ids_from_label_files'] =             Path(local_paths['dataset_dir'], 'ids_from_label_files_backup_dir.txt')

local_paths['yolo_data_cfg'] =                    Path(local_paths['dataset_dir'], 'yolo_data_cfg.yaml')
# local_paths['all_paths'] =                        Path(local_paths['dataset_dir'], 'all.txt')
# local_paths['train_paths'] =                      Path(local_paths['dataset_dir'], 'train.txt')
# local_paths['val_paths'] =                        Path(local_paths['dataset_dir'], 'val.txt')
# local_paths['test_paths'] =                       Path(local_paths['dataset_dir'], 'test.txt')

# local_paths['image_label_dir_crop'] =             Path(local_paths['dataset_dir'], 'image_label_1_crop')
local_paths['yolo_data_cfg_crop'] =               Path(local_paths['dataset_dir'], 'yolo_data_cfg_1_crop.yaml')
# local_paths['train_paths_crop'] =                 Path(local_paths['dataset_dir'], 'train_1_crop.txt')

## install ultralytics

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

## utils

In [ ]:
class Run():
    def __init__(self, run_name, project):
        self.train_run_name = f'train_{run_name}'
        self.val_run_name = f'val_{run_name}'
        self.run_path = Path(project, self.train_run_name)
        self.__best_weight_path = Path(self.run_path, 'weights', 'best.pt')
        self.__last_weight_path = Path(self.run_path, 'weights', 'last.pt')

    def get_best_weight_path(self):
        if self.run_path.is_dir():
            return self.__best_weight_path
        else:
            raise FileNotFoundError(f'The run {self.train_run_name} does not exist.')

    def get_last_weight_path(self):
        if self.run_path.is_dir():
            return self.__last_weight_path
        else:
            raise FileNotFoundError(f'The run {self.train_run_name} does not exist.')

## define runs

In [ ]:
yolo_default_augmentations = Run('default_augmentations_', paths['yolo_output_dir'])
yolo_no_augmentations = Run('no_augmentations_', paths['yolo_output_dir'])
yolo_1_crop = Run('1_crop_', paths['yolo_output_dir'])

# Get dataset

In [ ]:
shutil.copytree(paths['dataset_dir'], local_paths['dataset_dir'], dirs_exist_ok=True)

PosixPath('/content/dataset_v2_ALL')

In [ ]:
with tarfile.open(paths['processed_dataset_compressed'], 'r:gz') as tar_file:
    tar_file.extractall(local_paths['dataset_dir'])
    print(f"Extracted {paths['processed_dataset_compressed']} to {local_paths['dataset_dir']}")

Extracted /content/drive/MyDrive/fiber_optic_box_inspection/dataset_v2_ALL/processed_dataset.tar.gz to /content/dataset_v2_ALL


# GPU check

In [ ]:
!nvidia-smi

Sun Jun 29 15:23:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'GPU is available! Using device: {device}')
    print(f'GPU Name: {torch.cuda.get_device_name(0)}')
    print(f'Number of GPUs: {torch.cuda.device_count()}')
    print(f'Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
else:
    device = torch.device('cpu')
    print('No GPU available, using CPU.')


GPU is available! Using device: cuda
GPU Name: NVIDIA A100-SXM4-40GB
Number of GPUs: 1
Total Memory: 42.47 GB


# runs

## default augmentations

### train

In [ ]:
model = YOLO('yolov8m.pt')
results = model.train(
    data=local_paths['yolo_data_cfg'], # todo
    epochs=300,
    patience=5,
    batch=32,
    imgsz=IMGSZ,
    save_period=10,
    cache='disk',
    device=0,
    workers=16,
    project=paths['yolo_output_dir'],
    name=yolo_default_augmentations.train_run_name, # todo
    save_conf=True,
    # augmentation
    hsv_h=0.015,         # default 0.015
    hsv_s=0.7,           # default 0.7
    hsv_v=0.4,           # default 0.4
    degrees=0.0,         # default 0.0
    translate=0.1,       # default 0.1
    scale=0.5,           # default 0.5
    shear=0.0,           # default 0.0
    perspective=0.0,     # default 0.0
    flipud=0.0,          # default 0.0
    fliplr=0.5,          # default 0.5
    bgr=0.0,             # default 0.0
    mosaic=1.0,          # default 1.0
    mixup=0.0,           # default 0.0
    cutmix=0.0,          # default 0.0
)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_v2_ALL/yolo_data_cfg.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_default_augmentations_, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, persp

train: Scanning /content/dataset_v2_ALL/image_label... 7858 images, 466 backgrounds, 0 corrupt: 100%|██████████| 7858/7858 [00:05<00:00, 1494.88it/s]

train: /content/dataset_v2_ALL/image_label/2022_11_03_15_29_32-HNIP456107HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_10_14_59_59-HNIP120595HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_11_09_10_09-PYNP016064HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_11_09_59_56-BGGP089061HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_14_16_04_38-HNIP548102HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_16_08_43_48-HDGP141064HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_16_10_35_33-GLIP038152HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_16_15_40_51-HBHP012080HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_17_15_10_14-TNNP063261HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_A

train: New cache created: /content/dataset_v2_ALL/image_label.cache


train: Caching images (18.0GB Disk): 100%|██████████| 7858/7858 [00:51<00:00, 153.68it/s]


...

                   all       1684      17728      0.708      0.533      0.538      0.361

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     34/300      28.2G     0.9588     0.6567      1.221        384        960: 100%|██████████| 246/246 [01:53<00:00,  2.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:10<00:00,  2.52it/s]


                   all       1684      17728      0.653      0.558      0.548      0.368
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 29, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

34 epochs completed in 1.178 hours.
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_default_augmentations_/weights/last.pt, 52.2MB
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_default_augmentations_/weights/best.pt, 52.2MB

Validating /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_default_augmentations_/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,939,927 parameters, 0 gradients, 79.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:14<00:00,  1.85it/s]


                   all       1684      17728      0.738      0.531      0.557      0.377
                   1.0        146        146      0.722      0.918      0.898      0.787
                   1.1         67         67      0.636       0.73      0.759      0.578
                   1.2         27         27      0.381      0.778      0.456      0.345
                   1.7         15         15      0.259      0.374      0.385      0.319
                   2.0        255        256      0.915      0.953      0.967      0.854
                   2.3          2          2          1          0          0          0
                   3.0        245        245      0.921      0.935      0.958      0.627
                   3.1          1          1          0          0     0.0765     0.0612
                   5.0        240        478      0.947      0.969      0.975       0.67
                   5.1          3          3          0          0     0.0679     0.0225
                   6.

### resume train

In [ ]:
model = YOLO(yolo_default_augmentations.get_last_weight_path()) # todo
results = model.train(resume=True)

### val

In [ ]:
model = YOLO(yolo_default_augmentations.get_best_weight_path()) # todo
metrics = model.val(
    data=local_paths['yolo_data_cfg'],
    batch=32,
    imgsz=IMGSZ,
    save_json=True,
    device=0,
    plots=True,
    project=paths['yolo_output_dir'],
    name=yolo_default_augmentations.val_run_name, #todo
    verbose=True,
    save_txt=True,
    save_conf=True,
    workers=16,
)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,939,927 parameters, 0 gradients, 79.2 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 14.0MB/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3006.8±842.5 MB/s, size: 115.4 KB)



val: Scanning /content/dataset_v2_ALL/image_label... 1684 images, 101 backgrounds, 0 corrupt: 100%|██████████| 1684/1684 [00:01<00:00, 1352.46it/s]

val: /content/dataset_v2_ALL/image_label/2022_11_21_09_08_32-TNNP069058HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_11_21_16_32_02-HBHP012062HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_12_07_16_39_07-QNHP088205HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_12_09_10_16_40-TBHP044214HO.jpg: 1 duplicate labels removed
val: New cache created: /content/dataset_v2_ALL/image_label.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [01:40<00:00,  1.90s/it]


                   all       1684      17728      0.737      0.532      0.556      0.376
                   1.0        146        146       0.72      0.904      0.898      0.788
                   1.1         67         67      0.636      0.729      0.757      0.581
                   1.2         27         27      0.382      0.778      0.469      0.355
                   1.7         15         15      0.257      0.369      0.383       0.32
                   2.0        255        256      0.915      0.953      0.966      0.854
                   2.3          2          2          1          0          0          0
                   3.0        245        245      0.917      0.935      0.952      0.629
                   3.1          1          1          0          0     0.0765     0.0612
                   5.0        240        478      0.947      0.966      0.974      0.669
                   5.1          3          3          0          0     0.0678     0.0224
                   6.

## no augmentations

### train

In [ ]:
model = YOLO('yolov8m.pt')
results = model.train(
    data=local_paths['yolo_data_cfg'], # todo
    epochs=300,
    patience=5,
    batch=32,
    imgsz=IMGSZ,
    save_period=10,
    cache='disk',
    device=0,
    workers=16,
    project=paths['yolo_output_dir'],
    name=yolo_no_augmentations.train_run_name, # todo
    save_conf=True,
    # augmentation
    hsv_h=0.0,           # default 0.015
    hsv_s=0.0,           # default 0.7
    hsv_v=0.0,           # default 0.4
    degrees=0.0,         # default 0.0
    translate=0.0,       # default 0.1
    scale=0.0,           # default 0.5
    shear=0.0,           # default 0.0
    perspective=0.0,     # default 0.0
    flipud=0.0,          # default 0.0
    fliplr=0.0,          # default 0.5
    bgr=0.0,             # default 0.0
    mosaic=0.0,          # default 1.0
    mixup=0.0,           # default 0.0
    cutmix=0.0,          # default 0.0
)

100%|██████████| 49.7M/49.7M [00:00<00:00, 200MB/s]


Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_v2_ALL/yolo_data_cfg.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train_no_augmentations_, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=

100%|██████████| 5.35M/5.35M [00:00<00:00, 67.5MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2452.1±674.4 MB/s, size: 100.8 KB)


train: Scanning /content/dataset_v2_ALL/image_label... 7858 images, 466 backgrounds, 0 corrupt: 100%|██████████| 7858/7858 [00:05<00:00, 1456.14it/s]

...

                   all       1684      17728      0.652      0.497      0.494      0.328

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     19/300      27.2G     0.6131     0.3901     0.9895        175        960: 100%|██████████| 246/246 [01:48<00:00,  2.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:10<00:00,  2.55it/s]


                   all       1684      17728      0.562      0.532      0.496      0.331
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

19 epochs completed in 0.638 hours.
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_no_augmentations_/weights/last.pt, 52.2MB
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_no_augmentations_/weights/best.pt, 52.2MB

Validating /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_no_augmentations_/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,939,927 parameters, 0 gradients, 79.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:13<00:00,  2.05it/s]


                   all       1684      17728       0.62      0.521      0.501      0.332
                   1.0        146        146      0.671      0.945        0.8      0.683
                   1.1         67         67      0.746      0.483      0.653      0.495
                   1.2         27         27       0.33      0.556      0.404      0.297
                   1.7         15         15      0.279      0.285      0.318      0.239
                   2.0        255        256      0.917      0.961      0.955      0.825
                   2.3          2          2          0          0          0          0
                   3.0        245        245      0.916      0.922      0.925      0.584
                   3.1          1          1          1          0     0.0829      0.058
                   5.0        240        478      0.946      0.955      0.963      0.657
                   5.1          3          3          0          0       0.02     0.0116
                   6.

### resume train

In [ ]:
model = YOLO(yolo_no_augmentations.get_last_weight_path()) # todo
results = model.train(resume=True)

### val

In [ ]:
model = YOLO(yolo_no_augmentations.get_best_weight_path()) # todo
metrics = model.val(
    data=local_paths['yolo_data_cfg'],
    batch=32,
    imgsz=IMGSZ,
    save_json=True,
    device=0,
    plots=True,
    project=paths['yolo_output_dir'],
    name=yolo_no_augmentations.val_run_name, #todo
    verbose=True,
    save_txt=True,
    save_conf=True,
    workers=16,
)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,939,927 parameters, 0 gradients, 79.2 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 124MB/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3034.8±660.4 MB/s, size: 120.3 KB)



val: Scanning /content/dataset_v2_ALL/image_label... 1684 images, 101 backgrounds, 0 corrupt: 100%|██████████| 1684/1684 [00:01<00:00, 1461.66it/s]

val: /content/dataset_v2_ALL/image_label/2022_11_21_09_08_32-TNNP069058HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_11_21_16_32_02-HBHP012062HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_12_07_16_39_07-QNHP088205HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_12_09_10_16_40-TBHP044214HO.jpg: 1 duplicate labels removed
val: New cache created: /content/dataset_v2_ALL/image_label.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [01:23<00:00,  1.58s/it]


                   all       1684      17728       0.62       0.52      0.501      0.332
                   1.0        146        146      0.676      0.942        0.8      0.681
                   1.1         67         67      0.746      0.481      0.656      0.496
                   1.2         27         27      0.352      0.563      0.418      0.302
                   1.7         15         15      0.281      0.267      0.326      0.243
                   2.0        255        256      0.917      0.955      0.955      0.826
                   2.3          2          2          0          0          0          0
                   3.0        245        245      0.924      0.927      0.933      0.588
                   3.1          1          1          1          0     0.0905     0.0633
                   5.0        240        478      0.944      0.955      0.962      0.656
                   5.1          3          3          0          0       0.02     0.0116
                   6.

## 1 crop

### train

In [ ]:
model = YOLO('yolov8m.pt')
results = model.train(
    data=local_paths['yolo_data_cfg_crop'], # todo
    epochs=300,
    patience=5,
    batch=32,
    imgsz=IMGSZ,
    save_period=10,
    cache='disk',
    device=0,
    workers=16,
    project=paths['yolo_output_dir'],
    name=yolo_1_crop.train_run_name, # todo
    save_conf=True,
    # augmentation
    hsv_h=0.0,           # default 0.015
    hsv_s=0.0,           # default 0.7
    hsv_v=0.0,           # default 0.4
    degrees=0.0,         # default 0.0
    translate=0.0,       # default 0.1
    scale=0.0,           # default 0.5
    shear=0.0,           # default 0.0
    perspective=0.0,     # default 0.0
    flipud=0.0,          # default 0.0
    fliplr=0.0,          # default 0.5
    bgr=0.0,             # default 0.0
    mosaic=0.0,          # default 1.0
    mixup=0.0,           # default 0.0
    cutmix=0.0,          # default 0.0
)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_v2_ALL/yolo_data_cfg_1_crop.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train_1_crop_, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0

train: Scanning /content/dataset_v2_ALL/image_label... 11787 images, 704 backgrounds, 0 corrupt: 100%|██████████| 11787/11787 [00:11<00:00, 1037.59it/s]

train: /content/dataset_v2_ALL/image_label/2022_11_03_15_29_32-HNIP456107HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_10_14_59_59-HNIP120595HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_11_09_10_09-PYNP016064HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_11_09_59_56-BGGP089061HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_14_16_04_38-HNIP548102HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_16_08_43_48-HDGP141064HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_16_10_35_33-GLIP038152HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_16_15_40_51-HBHP012080HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_ALL/image_label/2022_11_17_15_10_14-TNNP063261HO.jpg: 1 duplicate labels removed
train: /content/dataset_v2_A

train: New cache created: /content/dataset_v2_ALL/image_label.cache


train: Caching images (28.1GB Disk): 100%|██████████| 11787/11787 [00:28<00:00, 413.30it/s]

...

                   all       1684      17728      0.595      0.531       0.51      0.339

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     22/300      27.3G     0.5736     0.3505     0.9667        117        960: 100%|██████████| 369/369 [02:44<00:00,  2.24it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:10<00:00,  2.51it/s]


                   all       1684      17728      0.609      0.495        0.5      0.335
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 17, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

22 epochs completed in 1.077 hours.
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_1_crop_/weights/last.pt, 52.2MB
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_1_crop_/weights/best.pt, 52.2MB

Validating /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/train_1_crop_/weights/best.pt...
Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,939,927 parameters, 0 gradients, 79.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:24<00:00,  1.11it/s]


                   all       1684      17728       0.57      0.541      0.516      0.342
                   1.0        146        146      0.685      0.894      0.827      0.704
                   1.1         67         67      0.636      0.731      0.679       0.51
                   1.2         27         27      0.421      0.592      0.437      0.302
                   1.7         15         15      0.125      0.267      0.172      0.141
                   2.0        255        256      0.914      0.945       0.93      0.812
                   2.3          2          2          0          0          0          0
                   3.0        245        245      0.898      0.914      0.914      0.584
                   3.1          1          1          0          0          0          0
                   5.0        240        478      0.943      0.946       0.96      0.667
                   5.1          3          3          0          0     0.0415     0.0271
                   6.

### resume train

In [ ]:
model = YOLO(yolo_1_crop.get_last_weight_path()) # todo
results = model.train(resume=True)

### val

In [ ]:
model = YOLO(yolo_1_crop.get_best_weight_path()) # todo
metrics = model.val(
    data=local_paths['yolo_data_cfg_crop'],
    batch=32,
    imgsz=IMGSZ,
    save_json=True,
    device=0,
    plots=True,
    project=paths['yolo_output_dir'],
    name=yolo_1_crop.val_run_name, # todo
    verbose=True,
    save_txt=True,
    save_conf=True,
    workers=16,
)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,939,927 parameters, 0 gradients, 79.2 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3081.6±609.4 MB/s, size: 119.6 KB)


val: Scanning /content/dataset_v2_ALL/image_label.cache... 1684 images, 101 backgrounds, 0 corrupt: 100%|██████████| 1684/1684 [00:00<?, ?it/s]

val: /content/dataset_v2_ALL/image_label/2022_11_21_09_08_32-TNNP069058HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_11_21_16_32_02-HBHP012062HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_12_07_16_39_07-QNHP088205HO.jpg: 1 duplicate labels removed
val: /content/dataset_v2_ALL/image_label/2022_12_09_10_16_40-TBHP044214HO.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [01:04<00:00,  1.22s/it]


                   all       1684      17728      0.568      0.537      0.515       0.34
                   1.0        146        146      0.678      0.884      0.826      0.704
                   1.1         67         67      0.658      0.746      0.684      0.518
                   1.2         27         27      0.387      0.514      0.411      0.272
                   1.7         15         15      0.134      0.267      0.192      0.161
                   2.0        255        256      0.914      0.945      0.932      0.814
                   2.3          2          2          0          0          0          0
                   3.0        245        245      0.899      0.911      0.913      0.582
                   3.1          1          1          0          0          0          0
                   5.0        240        478       0.94      0.946      0.957      0.663
                   5.1          3          3          0          0     0.0392     0.0256
                   6.